# Placement of the flows on the four available "slots"

For every stock in the model check how many and which flows it has.
- per stock loop through the flows.
- if the stock is a destination in the flow then the flow is an inflow in the stock.
- if the stock is a dource in the flow then the flow is an outflow in the stock.
- check whether there are maximally four in- and outflows.
- place the flows on the up, right, down, left positions of the stock drawing.

In [76]:
from ElementParser import ElementParser

In [165]:
PlaceFlows('ASD_V2.mdl')

{<ModelObjects.Stock at 0x1042c6eb8>: [[],
  {'right': <ModelObjects.Flow at 0x1043b82b0>},
  {'right': 'out'}],
 <ModelObjects.Stock at 0x10427bfd0>: [[<ModelObjects.Flow at 0x1043b89b0>],
  {'left': <ModelObjects.Flow at 0x1043b89b0>,
   'right': <ModelObjects.Flow at 0x1043b8080>},
  {'left': 'in', 'right': 'out'}],
 <ModelObjects.Stock at 0x1042c6b70>: [[<ModelObjects.Flow at 0x1043b82b0>,
  {'left': <ModelObjects.Flow at 0x1043b82b0>,
   'right': <ModelObjects.Flow at 0x1043b89b0>,
   'up': <ModelObjects.Flow at 0x1043b8080>},
  {'left': 'in', 'right': 'out', 'up': 'in'}]}

In [158]:
# gets flows and stocks and places them according to whether they are in or out flows
# returns a dictionary: {stock:[inflows],[outflows],{placements},{in/out mapping}}
def PlaceFlows(model):
    parser = ElementParser(model)
    elements = parser.parse()
    stocks = getStocks(elements)
    flows = getFlows(elements)
    InOutFlows = getInOutFlows(stocks,flows)
    if checkNumberOfFlows(InOutFlows) == True:
        placedFlows = placeFlowsSigns(InOutFlows)
        return placedFlows
    else:
        print("The model has one or more stocks with more than four in or out flows")
    

In [160]:
# Create a list of stocks
def getStocks(elements):
    stocks = []
    for elem in elements:
        if type(elem) == ModelObjects.Stock:
            stocks.append(elem)
    return stocks

In [161]:
# Create a list of flows
def getFlows(elements):
    flows = []
    for elem in elements:
        if type(elem) == ModelObjects.Flow:
            flows.append(elem)
    return flows

In [162]:
# Make lists of in and outflows per stock in a dictionary
# returns: {stock: [inflows],[outflows]}
def getInOutFlows(stocks,flows):
    InOutFlows = {}
    for stock in stocks:
        inflows = []
        outflows = []
        inandout = []
        for flow in flows:
            if stock.name == flow.dst.name:
                inflows.append(flow)
            elif stock.name == flow.src.name:
                outflows.append(flow)
        inandout = [inflows,outflows]
        InOutFlows[stock] = inandout
    return InOutFlows

In [163]:
# Check whether any of the stocks in the model have more than 4 in- or outflows.
def checkNumberOfFlows(InOutFlows):
    for key in InOutFlows:
        if len(InOutFlows[key][0])+len(InOutFlows[key][1])>4:
            return False
        else:
            return True

In [164]:
# create a dictionary of a stock with flows placed at the right place: right, down, left, up.
# {stock: [inflows],[outflows],{right, down, left, up},{in,out}}
def placeFlowsSigns(InOutFlows):
    for key in InOutFlows:
        FlowPlace = {}
        FlowSign = {}
        for flow in InOutFlows[key][0]:
            if 'left' not in FlowPlace.keys():
                FlowPlace['left'] = flow
                FlowSign['left'] = 'in'
            elif 'up' not in FlowPlace.keys():
                FlowPlace['up'] = flow
                FlowSign['up'] = 'in'
            elif 'down' not in FlowPlace.keys():
                FlowPlace['down'] = flow
                FlowSign['down'] = 'in'
            elif 'right' not in FlowPlace.keys():
                FlowPlace['right'] = flow
                FlowSign['right'] = 'in'
        for flow in InOutFlows[key][1]:
            if 'right' not in FlowPlace.keys():
                FlowPlace['right'] = flow
                FlowSign['right'] = 'out'
            elif 'down' not in FlowPlace.keys():
                FlowPlace['down'] = flow
                FlowSign['down'] = 'out'
            elif 'up' not in FlowPlace.keys():
                FLowPlace['up'] = flow
                FLowSign['up'] = 'out'
            elif 'left' not in FlowPlace.keys():
                FlowPlace['left'] = flow
                FlowSign['left'] = 'out'
        InOutFlows[key].append(FlowPlace)
        InOutFlows[key].append(FlowSign)
    return InOutFlows
